# ReAct Behavior Analysis


## Objective and Success Criteria

Goal: map how tool registration, execution paths, and runner wrappers shape ReAct behavior in this repository.

Success criteria:
- Extract a structured tool inventory from `build_tool_list` in `src/fleet_rlm/react/tools.py`.
- Summarize lifecycle/state transitions in `RLMReActChatAgent` from `src/fleet_rlm/react/agent.py`.
- Map key parameter propagation across runner entrypoints in `src/fleet_rlm/runners.py`.
- Run deterministic heuristic checks without requiring Modal or API credentials.


## Setup

Load standard-library analysis utilities plus optional tabular/plotting helpers.


In [1]:
from __future__ import annotations

import ast
import json
from pathlib import Path

try:
    import pandas as pd
except Exception:
    pd = None

try:
    import matplotlib.pyplot as plt  # Optional; not required for this notebook.
except Exception:
    plt = None

Resolve repository root and target source paths. This cell is filesystem-only and credential-free.


In [ ]:
def find_repo_root(start: Path | None = None) -> Path:
    start = (start or Path.cwd()).resolve()
    for candidate in [start, *start.parents]:
        if (candidate / ".git").exists() and (candidate / "src").exists():
            return candidate
    raise FileNotFoundError("Could not locate repo root containing .git and src/.")


repo_root = find_repo_root()
paths = {
    "tools_py": repo_root / "src/fleet_rlm/react/tools.py",
    "agent_py": repo_root / "src/fleet_rlm/react/agent.py",
    "runners_py": repo_root / "src/fleet_rlm/runners.py",
}

missing = [name for name, path in paths.items() if not path.exists()]
if missing:
    raise FileNotFoundError(f"Missing expected source files: {missing}")

print(f"Repository root: {repo_root}")
paths

Repository root: /Volumes/Samsung-SSD-T7/Workspaces/Github/qredence/agent-framework/v0.5/_WORLD/_RLM/fleet-rlm-dspy


{'tools_py': PosixPath('/Volumes/Samsung-SSD-T7/Workspaces/Github/qredence/agent-framework/v0.5/_WORLD/_RLM/fleet-rlm-dspy/src/fleet_rlm/react/tools.py'),
 'agent_py': PosixPath('/Volumes/Samsung-SSD-T7/Workspaces/Github/qredence/agent-framework/v0.5/_WORLD/_RLM/fleet-rlm-dspy/src/fleet_rlm/react/agent.py'),
 'runners_py': PosixPath('/Volumes/Samsung-SSD-T7/Workspaces/Github/qredence/agent-framework/v0.5/_WORLD/_RLM/fleet-rlm-dspy/src/fleet_rlm/runners.py')}

## Source Snapshot

Read source files and produce compact metadata for orientation.


In [3]:
sources = {name: path.read_text(encoding="utf-8") for name, path in paths.items()}

snapshot_rows = []
for name, text in sources.items():
    snapshot_rows.append(
        {
            "source": name,
            "path": str(paths[name].relative_to(repo_root)),
            "line_count": len(text.splitlines()),
            "char_count": len(text),
        }
    )

if pd is not None:
    source_snapshot_df = (
        pd.DataFrame(snapshot_rows).sort_values("source").reset_index(drop=True)
    )
    display(source_snapshot_df)
else:
    source_snapshot_df = snapshot_rows
    print(json.dumps(snapshot_rows, indent=2))

,source,path,line_count,char_count
0,agent_py,src/fleet_rlm/react/agent.py,450,17199
1,runners_py,src/fleet_rlm/runners.py,797,26360
2,tools_py,src/fleet_rlm/react/tools.py,761,25788


## Tool Surface Extraction (`tools.py`)

Parse `build_tool_list` and inventory nested tool functions by signature/docstring.


In [4]:
tools_tree = ast.parse(sources["tools_py"])
build_tool_list_node = next(
    node
    for node in tools_tree.body
    if isinstance(node, ast.FunctionDef) and node.name == "build_tool_list"
)


def stringify_annotation(node: ast.AST | None) -> str:
    if node is None:
        return ""
    try:
        return ast.unparse(node)
    except Exception:
        return ""


def extract_params(fn: ast.FunctionDef) -> list[tuple[str, str]]:
    args = fn.args.posonlyargs + fn.args.args
    defaults = [None] * (len(args) - len(fn.args.defaults)) + list(fn.args.defaults)
    out: list[tuple[str, str]] = []
    for arg, default in zip(args, defaults):
        default_text = "" if default is None else stringify_annotation(default)
        out.append((arg.arg, default_text))

    for kwarg, default in zip(fn.args.kwonlyargs, fn.args.kw_defaults):
        default_text = "" if default is None else stringify_annotation(default)
        out.append((kwarg.arg, default_text))
    return out


tool_inventory_rows = []
for node in build_tool_list_node.body:
    if not isinstance(node, ast.FunctionDef):
        continue
    params = extract_params(node)
    doc = ast.get_docstring(node) or ""
    tool_inventory_rows.append(
        {
            "tool_name": node.name,
            "param_count": len(params),
            "params": ", ".join(
                f"{name}={default}" if default else name for name, default in params
            ),
            "return_annotation": stringify_annotation(node.returns),
            "doc_first_line": doc.strip().splitlines()[0] if doc.strip() else "",
        }
    )

if pd is not None:
    tool_inventory_df = (
        pd.DataFrame(tool_inventory_rows)
        .sort_values("tool_name")
        .reset_index(drop=True)
    )
    display(tool_inventory_df)
else:
    tool_inventory_df = tool_inventory_rows
    print(json.dumps(tool_inventory_rows, indent=2))

,tool_name,param_count,params,return_annotation,doc_first_line
0,analyze_long_document,3,"query, alias='active', include_trajectory=True","dict[str, Any]",Analyze a long document with the AnalyzeLongDo...
1,chunk_host,5,"strategy, alias='active', size=200000, overlap...","dict[str, Any]",Chunk document on host using size/headers/time...
2,chunk_sandbox,6,"strategy, variable_name='active_document', buf...","dict[str, Any]",Chunk the active document inside sandbox and s...
3,clear_buffer,1,name='',"dict[str, Any]",Clear one sandbox buffer (or all buffers when ...
4,extract_from_logs,3,"query, alias='active', include_trajectory=True","dict[str, Any]",Extract structured patterns from log text via ...
5,find_files,3,"pattern, path='.', include=''","dict[str, Any]",Search file contents on the host using regex p...
6,list_documents,0,,"dict[str, Any]",List loaded document aliases and active docume...
7,list_files,2,"path='.', pattern='**/*'","dict[str, Any]",List files on the host filesystem matching a g...
8,load_document,2,"path, alias='active'","dict[str, Any]",Load a text document from host filesystem into...
9,load_text_from_volume,2,"path, alias='active'","dict[str, Any]",Load text from Modal Volume into host-side doc...


Preview only a few tool rows to keep output concise.


In [5]:
preview_count = 6
if pd is not None:
    display(tool_inventory_df.head(preview_count))
else:
    print(json.dumps(tool_inventory_rows[:preview_count], indent=2))

,tool_name,param_count,params,return_annotation,doc_first_line
0,analyze_long_document,3,"query, alias='active', include_trajectory=True","dict[str, Any]",Analyze a long document with the AnalyzeLongDo...
1,chunk_host,5,"strategy, alias='active', size=200000, overlap...","dict[str, Any]",Chunk document on host using size/headers/time...
2,chunk_sandbox,6,"strategy, variable_name='active_document', buf...","dict[str, Any]",Chunk the active document inside sandbox and s...
3,clear_buffer,1,name='',"dict[str, Any]",Clear one sandbox buffer (or all buffers when ...
4,extract_from_logs,3,"query, alias='active', include_trajectory=True","dict[str, Any]",Extract structured patterns from log text via ...
5,find_files,3,"pattern, path='.', include=''","dict[str, Any]",Search file contents on the host using regex p...


## Agent Lifecycle & State (`agent.py`)

Extract class methods and summarize lifecycle-relevant state transitions.


In [6]:
agent_tree = ast.parse(sources["agent_py"])
agent_class = next(
    node
    for node in agent_tree.body
    if isinstance(node, ast.ClassDef) and node.name == "RLMReActChatAgent"
)

agent_methods = [node for node in agent_class.body if isinstance(node, ast.FunctionDef)]
method_lookup = {node.name: node for node in agent_methods}
agent_lines = sources["agent_py"].splitlines()


def method_source(name: str) -> str:
    node = method_lookup[name]
    start = node.lineno - 1
    end = node.end_lineno
    return "\n".join(agent_lines[start:end])


method_rows = []
for name in [
    "_set_document",
    "start",
    "chat_turn",
    "achat_turn",
    "shutdown",
    "reset",
    "_build_agent",
]:
    node = method_lookup.get(name)
    if node is None:
        continue
    doc = ast.get_docstring(node) or ""
    method_rows.append(
        {
            "method": name,
            "line": node.lineno,
            "doc_first_line": doc.strip().splitlines()[0] if doc.strip() else "",
        }
    )

if pd is not None:
    agent_method_df = (
        pd.DataFrame(method_rows).sort_values("line").reset_index(drop=True)
    )
    display(agent_method_df)
else:
    agent_method_df = method_rows
    print(json.dumps(method_rows, indent=2))

,method,line,doc_first_line
0,_set_document,99,Set document with LRU eviction if needed.
1,start,136,Start the underlying Modal interpreter session...
2,shutdown,143,Shutdown the interpreter and mark this agent s...
3,reset,148,Reset chat history and (optionally) sandbox bu...
4,chat_turn,167,Process one interactive chat turn through the ...
5,_build_agent,432,


Build a compact transition summary for `start -> chat_turn -> shutdown` and `reset` with direct evidence snippets.


In [13]:
transition_rows = [
    {
        "flow": "start",
        "expectation": "Starts interpreter only once and sets _started=True.",
        "evidence_found": "if self._started" in method_source("start")
        and "self.interpreter.start()" in method_source("start")
        and "self._started = True" in method_source("start"),
    },
    {
        "flow": "chat_turn",
        "expectation": "Validates non-empty message, starts session, appends history.",
        "evidence_found": "if not message or not message.strip()"
        in method_source("chat_turn")
        and "self.start()" in method_source("chat_turn")
        and "self._append_history(" in method_source("chat_turn"),
    },
    {
        "flow": "shutdown",
        "expectation": "Shuts down interpreter and flips _started=False.",
        "evidence_found": "self.interpreter.shutdown()" in method_source("shutdown")
        and "self._started = False" in method_source("shutdown"),
    },
    {
        "flow": "reset",
        "expectation": "Clears history and optionally clears sandbox buffers.",
        "evidence_found": "self.history = dspy.History(messages=[])"
        in method_source("reset")
        and "if clear_sandbox_buffers" in method_source("reset")
        and 'if getattr(tool, "__name__", None) == "clear_buffer"'
        in method_source("reset"),
    },
]

if pd is not None:
    transition_df = pd.DataFrame(transition_rows)
    display(transition_df)
else:
    transition_df = transition_rows
    print(json.dumps(transition_rows, indent=2))

,flow,expectation,evidence_found
0,start,Starts interpreter only once and sets _started...,True
1,chat_turn,"Validates non-empty message, starts session, a...",True
2,shutdown,Shuts down interpreter and flips _started=False.,True
3,reset,Clears history and optionally clears sandbox b...,True


## Runner Entry Point Mapping (`runners.py`)

Parse the three entrypoints and compare their parameter surfaces.


In [7]:
runners_tree = ast.parse(sources["runners_py"])
runner_targets = [
    "build_react_chat_agent",
    "run_react_chat_once",
    "arun_react_chat_once",
]
runner_nodes = {
    node.name: node
    for node in runners_tree.body
    if isinstance(node, (ast.FunctionDef, ast.AsyncFunctionDef))
    and node.name in runner_targets
}


def extract_signature_rows(
    func_name: str, node: ast.FunctionDef | ast.AsyncFunctionDef
) -> list[dict[str, str]]:
    rows: list[dict[str, str]] = []

    positional = node.args.posonlyargs + node.args.args
    pos_defaults = [None] * (len(positional) - len(node.args.defaults)) + list(
        node.args.defaults
    )
    for arg, default in zip(positional, pos_defaults):
        rows.append(
            {
                "function": func_name,
                "parameter": arg.arg,
                "kind": "positional_or_keyword",
                "default": stringify_annotation(default) if default is not None else "",
            }
        )

    for kwarg, default in zip(node.args.kwonlyargs, node.args.kw_defaults):
        rows.append(
            {
                "function": func_name,
                "parameter": kwarg.arg,
                "kind": "keyword_only",
                "default": stringify_annotation(default) if default is not None else "",
            }
        )

    return rows


runner_signature_rows = []
for fn_name in runner_targets:
    runner_signature_rows.extend(extract_signature_rows(fn_name, runner_nodes[fn_name]))

if pd is not None:
    runner_signature_df = pd.DataFrame(runner_signature_rows)
    display(runner_signature_df)
else:
    runner_signature_df = runner_signature_rows
    print(json.dumps(runner_signature_rows, indent=2))

,function,parameter,kind,default
0,build_react_chat_agent,docs_path,keyword_only,None
1,build_react_chat_agent,react_max_iters,keyword_only,10
2,build_react_chat_agent,rlm_max_iterations,keyword_only,30
3,build_react_chat_agent,rlm_max_llm_calls,keyword_only,50
4,build_react_chat_agent,timeout,keyword_only,900
5,build_react_chat_agent,secret_name,keyword_only,'LITELLM'
6,build_react_chat_agent,volume_name,keyword_only,None
7,build_react_chat_agent,verbose,keyword_only,False
8,build_react_chat_agent,history_max_turns,keyword_only,None
9,build_react_chat_agent,extra_tools,keyword_only,None


Summarize propagation for key parameters across all three runner entrypoints.


In [8]:
tracked_params = [
    "docs_path",
    "react_max_iters",
    "rlm_max_iterations",
    "rlm_max_llm_calls",
    "timeout",
    "secret_name",
    "volume_name",
    "verbose",
    "include_trajectory",
    "env_file",
    "planner_lm",
]

runner_param_sets = {
    fn_name: {
        row["parameter"] for row in runner_signature_rows if row["function"] == fn_name
    }
    for fn_name in runner_targets
}

propagation_rows = []
for param in tracked_params:
    row = {"parameter": param}
    for fn_name in runner_targets:
        row[fn_name] = param in runner_param_sets[fn_name]
    propagation_rows.append(row)

if pd is not None:
    propagation_df = pd.DataFrame(propagation_rows)
    display(propagation_df)
else:
    propagation_df = propagation_rows
    print(json.dumps(propagation_rows, indent=2))

,parameter,build_react_chat_agent,run_react_chat_once,arun_react_chat_once
0,docs_path,True,True,True
1,react_max_iters,True,True,True
2,rlm_max_iterations,True,True,True
3,rlm_max_llm_calls,True,True,True
4,timeout,True,True,True
5,secret_name,True,True,True
6,volume_name,True,True,True
7,verbose,True,True,True
8,include_trajectory,False,True,True
9,env_file,True,True,True


## Behavioral Heuristics

Run deterministic rule checks against source text for key behavioral expectations.


In [9]:
heuristic_rows = [
    {
        "check": "empty-message guard present",
        "status": "PASS"
        if "if not message or not message.strip()" in sources["agent_py"]
        else "FAIL",
        "evidence": "chat_turn/achat_turn input validation guard",
    },
    {
        "check": "trajectory include/remove behavior",
        "status": "PASS"
        if (
            "if not include_trajectory:" in sources["runners_py"]
            and 'result.pop("trajectory", None)' in sources["runners_py"]
        )
        else "FAIL",
        "evidence": "run_react_chat_once + arun_react_chat_once trajectory toggle",
    },
    {
        "check": "document cache eviction behavior noted",
        "status": "PASS"
        if (
            "len(self._document_cache) >= self._max_documents" in sources["agent_py"]
            and "oldest = self._document_access_order.pop(0)" in sources["agent_py"]
        )
        else "FAIL",
        "evidence": "_set_document LRU eviction path",
    },
    {
        "check": "tool list extension via extra_tools",
        "status": "PASS"
        if (
            "if extra_tools:" in sources["tools_py"]
            and "tools.extend(extra_tools)" in sources["tools_py"]
        )
        else "FAIL",
        "evidence": "build_tool_list extension branch",
    },
]

if pd is not None:
    heuristic_df = pd.DataFrame(heuristic_rows)
    display(heuristic_df)
else:
    heuristic_df = heuristic_rows
    print(json.dumps(heuristic_rows, indent=2))

,check,status,evidence
0,empty-message guard present,PASS,chat_turn/achat_turn input validation guard
1,trajectory include/remove behavior,PASS,run_react_chat_once + arun_react_chat_once tra...
2,document cache eviction behavior noted,PASS,_set_document LRU eviction path
3,tool list extension via extra_tools,PASS,build_tool_list extension branch


## Optional Deep-Dive Cells (No Credentials Required)

The snippet below is intentionally non-executing by default. It shows how to run live calls only when env checks pass.


In [10]:
optional_live_example = """
# Optional live-run sketch (disabled by default)
# from fleet_rlm.runners import run_react_chat_once
# import os

# required = ["DSPY_LM_MODEL", "DSPY_LLM_API_KEY"]
# missing = [k for k in required if not os.getenv(k)]
# if missing:
#     print(f"Skipping live run, missing env vars: {missing}")
# else:
#     result = run_react_chat_once(
#         message="Summarize the active document and list key tool actions.",
#         docs_path="README.md",
#         include_trajectory=True,
#     )
#     print(result["assistant_response"])
#     print(f"Trajectory keys: {list(result.get('trajectory', {}).keys())}")
""".strip()

print(optional_live_example)

# Optional live-run sketch (disabled by default)
# from fleet_rlm.runners import run_react_chat_once
# import os

# required = ["DSPY_LM_MODEL", "DSPY_LLM_API_KEY"]
# missing = [k for k in required if not os.getenv(k)]
# if missing:
#     print(f"Skipping live run, missing env vars: {missing}")
# else:
#     result = run_react_chat_once(
#         message="Summarize the active document and list key tool actions.",
#         docs_path="README.md",
#         include_trajectory=True,
#     )
#     print(result["assistant_response"])
#     print(f"Trajectory keys: {list(result.get('trajectory', {}).keys())}")


## Conclusions

Generate concise conclusions directly from extracted artifacts.


In [11]:
pass_count = sum(1 for row in heuristic_rows if row["status"] == "PASS")
fail_count = sum(1 for row in heuristic_rows if row["status"] == "FAIL")

conclusion_lines = [
    f"Identified {len(tool_inventory_rows)} nested tools inside build_tool_list.",
    f"Validated lifecycle flow evidence for {sum(1 for row in transition_rows if row['evidence_found'])}/{len(transition_rows)} transition checks.",
    f"Tracked {len(tracked_params)} key parameters across {len(runner_targets)} runner entrypoints.",
    f"Heuristic checks: {pass_count} PASS / {fail_count} FAIL.",
]

print("\n".join(f"- {line}" for line in conclusion_lines))

NameError: name 'transition_rows' is not defined

## Next Experiments

1. Add streaming-specific analysis for `iter_chat_turn_stream` and `aiter_chat_turn_stream` event parity.
2. Build an async/sync behavior parity notebook using synthetic prompts and mocked interpreter responses.
3. Create a lightweight regression notebook that snapshots tool signatures and flags drift across commits.
